# import funcs and load Data

In [ ]:
import numpy as np
import pandas as pd
from scipy import interpolate

import xarray as xr
import dask.array as da
from dask.distributed import Client

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches # for creating legend
import matplotlib.dates as mdates # converts datetime64 to datetime

import cartopy
import cartopy.crs as ccrs # for plotting
import cartopy.feature as cfeature # for map features
from cartopy.util import add_cyclic_point # for wrapping map fully - avoiding white line on 0 deg
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import matplotlib.dates as mdates # converts datetime64 to datetime
import matplotlib.gridspec as gridspec # to create grid-shaped combos of axes
from mpl_toolkits import mplot3d # 3d plotting tool
import cmocean # for nice oceanography colour pallettes

#import argopy
#from argopy import DataFetcher as ArgoDataFetcher # to load Argo ds directly

import os # for finding files

import gsw # for conversion functions

from tqdm.notebook import tqdm_notebook as tqdm
import glob # for downloading data
import sys # for path to functions

import seaborn as sns

sns.set(#font='Franklin Gothic Book',
        rc={
         'axes.axisbelow': False,
         'axes.edgecolor': 'Black',
         'axes.facecolor': 'w', 
                            # '#aeaeae',
         'axes.grid': False,
         'axes.labelcolor': 'k',
         'axes.spines.right': True,
         'axes.spines.top': True,
         'figure.facecolor': 'white',
         'lines.solid_capstyle': 'round',
         'patch.edgecolor': 'k',
         'patch.force_edgecolor': True,
         'text.color': 'k',
         'xtick.bottom': True,
         'xtick.color': 'k',
         'xtick.direction': 'out',
         'xtick.top': False,
         'ytick.color': 'k',
         'ytick.direction': 'out',
         'ytick.left': True,
         'ytick.right': False},
         font_scale=1)
mpl.rcParams["figure.titlesize"] = 20
mpl.rcParams["axes.titlesize"] = 20
mpl.rcParams["axes.labelsize"] = 12
mpl.rcParams["font.size"] = 10
mpl.rcParams["xtick.labelsize"] = 12
mpl.rcParams["ytick.labelsize"] = 12
mpl.rcParams["ytick.labelright"] = False

from warnings import filterwarnings as fw
fw('ignore')

In [ ]:
import importlib
#importlib.reload(sys.modules['load_data'])

path = '/home/theospira/notebooks/projects/WW_climatology'

import sys
sys.path.append(path+'/functions')
from plot_formatting import *
from inspection_funcs import boxplot
from smoothing_and_interp import *

from load_data import load_data
# load data
ds, ds_s, ds_ww, bth, ssh, si, szn = load_data(path+'/data/hydrographic_profiles/SO_1yr_clim_seasonal.nc')

# funcs

In [ ]:
# Define a function to create a figure displaying seasonal differences of a variable
def plot_seasonal_diff(var, ax, **kwargs):
    """
    Create a figure displaying seasonal differences of a variable.

    Parameters:
    -----------
    var : xarray.DataArray
        The input variable with seasonal data.

    ax : list of matplotlib.Axes
        A list of subplot axes where the seasonal differences will be plotted.

    kwargs : keyword arguments
        Additional keyword arguments to customize the plots.

    Returns:
    --------
    hmp : xarray.DataArray
        The result of the plot function.
    """
    crs = ccrs.PlateCarree()

    for i, a in enumerate(ax):
        hmp = (var.isel(season=i) - var.isel(season=(int(i - 1) % 4))).plot(x='lon', transform=crs, ax=a, **kwargs)

    return hmp

# Define a function to create a figure displaying the mean seasonal change of a variable
def plot_mean_seasonal_change(ax, var, var_lab, units, si, cmap, vmin, vmax, szn_vmax, szn_cmap, shrink=0.7, asp=12, fs=20,
                              set_ticks=False, ticks=[], set_ticks2=False, ticks2=[], si_x=15):
    """
    Create a figure displaying the mean seasonal change of a variable and sea ice contours.

    Parameters:
    -----------
    ax : list of matplotlib.Axes
        A list of subplot axes where the plots will be displayed.

    var : xarray.DataArray
        The input variable with seasonal data.

    var_lab : str
        The label for the variable.

    units : str
        The units of the variable.

    si : xarray.DataArray
        Sea ice data.

    si_x : int, optional
        Sea ice parameter.

    cmap : str
        The colormap for the plot.

    vmin : float
        The minimum value for the color scale.

    vmax : float
        The maximum value for the color scale.

    szn_vmax : float
        The maximum value for the seasonal change plot.

    szn_cmap : str
        The colormap for the seasonal change plot.

    shrink : float, optional
        The shrink factor for the colorbars.

    asp : int, optional
        The aspect ratio of the colorbars.

    fs : int, optional
        The fontsize for labels and colorbar labels.

    set_ticks : bool, optional
        Whether to set colorbar ticks.

    ticks : list, optional
        Custom colorbar ticks.

    set_ticks2 : bool, optional
        Whether to set colorbar ticks for the seasonal change plot.

    ticks2 : list, optional
        Custom colorbar ticks for the seasonal change plot.

    Returns:
    --------
    None
    """
    crs = ccrs.PlateCarree()

    # Plot the mean
    hmp = var.mean('season').plot(x='lon', transform=crs, ax=ax[0], add_colorbar=False, vmin=vmin, vmax=vmax, cmap=cmap)
    cb = fig.colorbar(hmp, ax=ax[0], shrink=shrink, aspect=asp, location='left')
    cb.set_label(label=var_lab + '\n(' + units + ')', fontsize=fs)
    if set_ticks:
        cb.set_ticks(ticks)

    # Plot the seasonal change
    hmp = plot_seasonal_diff(var, ax[1:5], vmin=-szn_vmax, vmax=szn_vmax, cmap=szn_cmap, add_colorbar=False)
    cb = fig.colorbar(hmp, ax=ax[1:5], shrink=shrink, aspect=asp)
    cb.set_label(label='Seasonal\nChange (' + units + ')', fontsize=fs)
    if set_ticks2:
        cb.set_ticks(ticks2)

    # Plot sea ice contour
    ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'), nx=20
                       ).plot(ax=ax[0], c='w', transform=crs, lw=1.25)
    for i, a in enumerate(ax[1:5]):
        ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=i), nx=20
                           ).plot(ax=a, c='w', transform=crs, lw=1.25)


# Define a function to create a figure displaying seasonal anomalies of a variable
def plot_seasonal_anomaly(var, ax, **kwargs):
    """
    Create a figure displaying seasonal anomalies of a variable.

    Parameters:
    -----------
    var : xarray.DataArray
        The input variable with seasonal data.

    ax : list of matplotlib.Axes
        A list of subplot axes where the seasonal anomalies will be plotted.

    kwargs : keyword arguments
        Additional keyword arguments to customize the plots.

    Returns:
    --------
    hmp : xarray.DataArray
        The result of the plot function.
    """
    crs = ccrs.PlateCarree()

    for i, a in enumerate(ax):
        hmp = (var.isel(season=i) - var.mean('season')).plot(x='lon', transform=crs, ax=a, **kwargs)

    return hmp

# Define a function to create a figure displaying the mean anomaly of a variable and sea ice contours
def plot_mean_anomaly(ax, var, var_lab, units, si, cmap, vmin, vmax, szn_vmax, szn_cmap, shrink=0.7, asp=12, fs=20,
                      set_ticks=False, ticks=[], set_ticks2=False, ticks2=[], si_x=15):
    """
    Create a figure displaying the mean anomaly of a variable and sea ice contours.

    Parameters:
    -----------
    ax : list of matplotlib.Axes
        A list of subplot axes where the plots will be displayed.

    var : xarray.DataArray
        The input variable with seasonal data.

    var_lab : str
        The label for the variable.

    units : str
        The units of the variable.

    si : xarray.DataArray
        Sea ice data.

    cmap : str
        The colormap for the plot.

    vmin : float
        The minimum value for the color scale.

    vmax : float
        The maximum value for the color scale.

    szn_vmax : float
        The maximum value for the seasonal anomaly plot.

    szn_cmap : str
        The colormap for the seasonal anomaly plot.

    shrink : float, optional
        The shrink factor for the colorbars.

    asp : int, optional
        The aspect ratio of the colorbars.

    fs : int, optional
        The fontsize for labels and colorbar labels.

    set_ticks : bool, optional
        Whether to set colorbar ticks.

    ticks : list, optional
        Custom colorbar ticks.

    set_ticks2 : bool, optional
        Whether to set colorbar ticks for the seasonal anomaly plot.

    ticks2 : list, optional
        Custom colorbar ticks for the seasonal anomaly plot.

    si_x : int, optional
        Sea ice parameter.

    Returns:
    --------
    None
    """
    crs = ccrs.PlateCarree()

    # Plot the mean
    hmp = var.mean('season').plot(x='lon', transform=crs, ax=ax[0], add_colorbar=False, vmin=vmin, vmax=vmax, cmap=cmap)
    cb = fig.colorbar(hmp, ax=ax[0], shrink=shrink, aspect=asp, location='left')
    cb.set_label(label=var_lab + '\n(' + units + ')', fontsize=fs)
    if set_ticks:
        cb.set_ticks(ticks)

    # Plot the anomaly
    hmp = plot_seasonal_anomaly(var, ax[1:5], vmin=-szn_vmax, vmax=szn_vmax, cmap=szn_cmap, add_colorbar=False)
    cb = fig.colorbar(hmp, ax=ax[1:5], shrink=shrink, aspect=asp)
    cb.set_label(label='Anomaly\n(' + units + ')', fontsize=fs)
    if set_ticks2:
        cb.set_ticks(ticks2)

    # Plot sea ice contour
    ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'), nx=20
                       ).plot(ax=ax[0], c='w', transform=crs, lw=1.25)
    for i, a in enumerate(ax[1:5]):
        ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=i), nx=20
                           ).plot(ax=a, c='w', transform=crs, lw=1.25)

# check some stats

In [ ]:
def stats_var(ds,var,decimals=2):
    if var == 'ww_n2_int':
        ds[var] = ds[var]*1e3
    if var == 'ww_ct':
        print(var+' mn: ',np.round(ds[var].mean(('lat','lon'),skipna=True).mean('season').data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).mean('season').data,decimals)),
                  str(np.round((ds[var]+273.15).std(('lat','lon'),skipna=True).mean('season').data/(
                                ds[var]+273.15).mean(('lat','lon'),skipna=True).mean('season').data * 100,decimals=decimals)) +'%')
        for i in range(4):
            print(var+' '+str(i+1)+' : ',np.round(ds[var].mean(('lat','lon'),skipna=True).isel(season=i).data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).isel(season=i).data,decimals)),
                  str(np.round((ds[var]+273.15).std(('lat','lon'),skipna=True).isel(season=i).data/(
                                ds[var]+273.15).mean(('lat','lon'),skipna=True).isel(season=i).data * 100,decimals=decimals)) +'%')
    else:
        print(var+' mn: ',np.round(ds[var].mean(('lat','lon'),skipna=True).mean('season').data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).mean('season').data,decimals)),
                  str(np.round((ds[var]).std(('lat','lon'),skipna=True).mean('season').data/(
                                ds[var]).mean(('lat','lon'),skipna=True).mean('season').data * 100,decimals=decimals)) +'%')
        for i in range(4):
            print(var+' '+str(i+1)+' : ',np.round(ds[var].mean(('lat','lon'),skipna=True).isel(season=i).data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).isel(season=i).data,decimals)),
                  str(np.round((ds[var]).std(('lat','lon'),skipna=True).isel(season=i).data/(
                                ds[var]).mean(('lat','lon'),skipna=True).isel(season=i).data * 100,decimals=decimals)) +'%')
    if var == 'ww_n2_int':
        ds[var] = ds[var]*1e-3

In [ ]:
def stats_var_median(ds,var,decimals=2):
    if var == 'ww_n2_int':
        ds[var] = ds[var]*1e3
    if var == 'ww_ct':
        print(var+' mn: ',np.round(ds[var].median(('lat','lon'),skipna=True).median('season').data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).median('season').data,decimals)),
                  str(np.round((ds[var]+273.15).std(('lat','lon'),skipna=True).median('season').data/(
                                ds[var]+273.15).median(('lat','lon'),skipna=True).median('season').data * 100,decimals=decimals)) +'%')
        for i in range(4):
            print(var+' '+str(i+1)+' : ',np.round(ds[var].median(('lat','lon'),skipna=True).isel(season=i).data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).isel(season=i).data,decimals)),
                  str(np.round((ds[var]+273.15).std(('lat','lon'),skipna=True).isel(season=i).data/(
                                ds[var]+273.15).median(('lat','lon'),skipna=True).isel(season=i).data * 100,decimals=decimals)) +'%')
    else:
        print(var+' mn: ',np.round(ds[var].median(('lat','lon'),skipna=True).median('season').data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).median('season').data,decimals)),
                  str(np.round((ds[var]).std(('lat','lon'),skipna=True).median('season').data/(
                                ds[var]).median(('lat','lon'),skipna=True).median('season').data * 100,decimals=decimals)) +'%')
        for i in range(4):
            print(var+' '+str(i+1)+' : ',np.round(ds[var].median(('lat','lon'),skipna=True).isel(season=i).data,decimals),
                  '± '+str(np.round(ds[var].std(('lat','lon'),skipna=True).isel(season=i).data,decimals)),
                  str(np.round((ds[var]).std(('lat','lon'),skipna=True).isel(season=i).data/(
                                ds[var]).median(('lat','lon'),skipna=True).isel(season=i).data * 100,decimals=decimals)) +'%')
    if var == 'ww_n2_int':
        ds[var] = ds[var]*1e-3

In [ ]:
def anom_stats(dsvar,szn,decimals):
    
    for i,s in enumerate(szn):
        var = (dsvar.isel(season=i) - dsvar.mean('season'))
        print(s,np.round(var.mean(skipna=True).data,decimals),
                      '± ', np.round(var.std(skipna=True).data,decimals),)

In [ ]:
vmax=2
ds_ww['ww_ct'].isel(season=0).plot(x='lon',vmin=-vmax,vmax=vmax,cmap='cmo.thermal',levels=int(vmax * 8 +1),figsize=(14,5))

(dsvar.isel(season=i) - dsvar.mean('season')).plot.contour(x='lon',levels=[-0.5],colors=['w'])

In [ ]:
var = ['ww_ct','ww_sa' ]

for v,d in zip(var,[2,2]):
    print('\n','entire SO ',v,)
    anom_stats(ds_ww[v],szn,decimals=d)

In [ ]:
for i in range(4):
    print(i, np.round(ds_ww['ww_sa'][i].median().values,2))

In [ ]:
for i in range(4):
    print(i, np.round(ds_ww['ww_sa'][i].mean().values,2))

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['ww_ct','ww_sa' ]

for v,d in zip(var,[1,2]):
    print(v)
    print('entire SO')
    anom_stats(ds_ww[v],szn,decimals=d)
    print('\n''NORTH OF SEA ICE')
    anom_stats(ds_ww.where(ds_ww.lat>ice)[v],szn,decimals=d)
    print('\n''SOUTH OF SEA ICE')
    anom_stats(ds_ww.where(ds_ww.lat<=ice)[v],szn,decimals=d)
    print('\n')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['ww_ct','ww_sa' ]

for v,d in zip(var,[1,2]):
    print(v)
    print('entire SO')
    stats_var(ds_ww,v,decimals=d)
    print('\n''NORTH OF SEA ICE')
    stats_var(ds_ww.where(ds_ww.lat>ice),v,decimals=d)
    print('\n''SOUTH OF SEA ICE')
    stats_var(ds_ww.where(ds_ww.lat<=ice),v,decimals=d)
    print('\n')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['mld' ]

for v,d in zip(var,[1,2]):
    print(v)
    print('entire SO')
    stats_var(ds_ww,v,decimals=d)
    print('\n''NORTH OF SEA ICE')
    stats_var(ds_ww.where(ds_ww.lat>ice),v,decimals=d)
    print('\n''SOUTH OF SEA ICE')
    stats_var(ds_ww.where(ds_ww.lat<=ice),v,decimals=d)
    print('\n')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['ww_ct','ww_sa' ]

for v,d in zip(var,[1,2]):
    print(v)
    print('entire SO')
    stats_var(ds_ww,v,decimals=d)
    print('\n''NORTH OF SEA ICE')
    stats_var(ds_ww.where(ds_ww.lat>ice),v,decimals=d)
    print('\n''SOUTH OF SEA ICE')
    stats_var(ds_ww.where(ds_ww.lat<=ice),v,decimals=d)
    print('\n')

In [ ]:
for i in range(4):
    print(szn[i], np.round(ds_ww.ww_sa[i].mean().data,2))

In [ ]:
np.round(ds_ww.ww_cd.reduce(np.nanstd).data,0)

In [ ]:
for i in range(4):
    print(szn[i], np.round(ds_ww.ww_cd[i].reduce(np.nanstd).data,0))

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['ww_ct','ww_sa' ]

for v,d in zip(var,[1,2]):
    print(v)
    print('entire SO')
    stats_var_(ds_ww,v,decimals=d)
    print('\n''NORTH OF SEA ICE')
    stats_var_(ds_ww.where(ds_ww.lat>ice),v,decimals=d)
    print('\n''SOUTH OF SEA ICE')
    stats_var_(ds_ww.where(ds_ww.lat<=ice),v,decimals=d)
    print('\n')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['ww_cd','thcc', ]

for v in var:
    print(v)
    print('entire SO')
    stats_var_median(ds_ww,v,decimals=0)
    print('\n''NORTH OF SEA ICE')
    stats_var_median(ds_ww.where(ds_ww.lat>ice),v,decimals=0)
    print('\n''SOUTH OF SEA ICE')
    stats_var_median(ds_ww.where(ds_ww.lat<=ice),v,decimals=0)
    print('\n')

In [ ]:
def warming_rate_n_of_sea_ice(ds_ww,var,ds_s):
    ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
    tmp = ds_ww[var].mean(('season','lon'))
    return tmp.where(tmp.lat>ice.mean(('season','lon')).mean()).diff('lat').mean()

In [ ]:
for var in ['ww_ct','ww_sa','ww_cd','thcc']:
    print(var, warming_rate_n_of_sea_ice(ds_ww,var,ds_s).data.round(2))

### check difference between anomaly and mean 

In [ ]:
dsvar = ds_ww.thcc
print('anomaly mean')
for i in range(4):
    var = (dsvar.isel(season=i) - dsvar.mean('season'))
    
    print(szn[i],var.mean().data.round(0),' ± ',var.std().data.round(0))

In [ ]:
dsvar = ds_ww.thcc
print('dt mean')
for i in range(4):
    var = (dsvar.isel(season=i) - dsvar.isel(season=(int(i - 1) % 4)))
    
    print(szn[i],var.mean().data.round(0),' ± ',var.std().data.round(0))

In [ ]:
var.plot(x='lon',vmin=-50,vmax=50,cmap='cmo.balance',figsize=(14,4))

In [ ]:
var.mean()

In [ ]:
var.std()

# fig 2: WW temp, sal and N2

In [ ]:
# Define a function to create a figure displaying seasonal differences of a variable
def plot_seasonal_diff(var, ax, **kwargs):
    """
    Create a figure displaying seasonal differences of a variable.

    Parameters:
    -----------
    var : xarray.DataArray
        The input variable with seasonal data.

    ax : list of matplotlib.Axes
        A list of subplot axes where the seasonal differences will be plotted.

    kwargs : keyword arguments
        Additional keyword arguments to customize the plots.

    Returns:
    --------
    hmp : xarray.DataArray
        The result of the plot function.
    """
    crs = ccrs.PlateCarree()

    for i, a in enumerate(ax):
        hmp = (var.isel(season=i) - var.isel(season=(int(i - 1) % 4))).plot(x='lon', transform=crs, ax=a, **kwargs)

    return hmp

# Define a function to create a figure displaying the mean seasonal change of a variable
def plot_mean_seasonal_change(ax, var, var_lab, units, si, cmap, vmin, vmax, szn_vmax, szn_cmap, shrink=0.7, asp=12, fs=20,
                              set_ticks=False, ticks=[], set_ticks2=False, ticks2=[], si_x=15):
    """
    Create a figure displaying the mean seasonal change of a variable and sea ice contours.

    Parameters:
    -----------
    ax : list of matplotlib.Axes
        A list of subplot axes where the plots will be displayed.

    var : xarray.DataArray
        The input variable with seasonal data.

    var_lab : str
        The label for the variable.

    units : str
        The units of the variable.

    si : xarray.DataArray
        Sea ice data.

    si_x : int, optional
        Sea ice parameter.

    cmap : str
        The colormap for the plot.

    vmin : float
        The minimum value for the color scale.

    vmax : float
        The maximum value for the color scale.

    szn_vmax : float
        The maximum value for the seasonal change plot.

    szn_cmap : str
        The colormap for the seasonal change plot.

    shrink : float, optional
        The shrink factor for the colorbars.

    asp : int, optional
        The aspect ratio of the colorbars.

    fs : int, optional
        The fontsize for labels and colorbar labels.

    set_ticks : bool, optional
        Whether to set colorbar ticks.

    ticks : list, optional
        Custom colorbar ticks.

    set_ticks2 : bool, optional
        Whether to set colorbar ticks for the seasonal change plot.

    ticks2 : list, optional
        Custom colorbar ticks for the seasonal change plot.

    Returns:
    --------
    None
    """
    crs = ccrs.PlateCarree()

    # Plot the mean
    hmp = var.mean('season').plot(x='lon', transform=crs, ax=ax[0], add_colorbar=False, vmin=vmin, vmax=vmax, cmap=cmap)
    cb = fig.colorbar(hmp, ax=ax[0], shrink=shrink, aspect=asp, location='left')
    cb.set_label(label=var_lab + '\n(' + units + ')', fontsize=fs)
    if set_ticks:
        cb.set_ticks(ticks)

    # Plot the seasonal change
    hmp = plot_seasonal_diff(var, ax[1:5], vmin=-szn_vmax, vmax=szn_vmax, cmap=szn_cmap, add_colorbar=False)
    cb = fig.colorbar(hmp, ax=ax[1:5], shrink=shrink, aspect=asp)
    cb.set_label(label='Seasonal\nChange (' + units + ')', fontsize=fs)
    if set_ticks2:
        cb.set_ticks(ticks2)

    # Plot sea ice contour
    ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'), nx=20
                       ).plot(ax=ax[0], c='w', transform=crs, lw=1.25)
    for i, a in enumerate(ax[1:5]):
        ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=i), nx=20
                           ).plot(ax=a, c='w', transform=crs, lw=1.25)


# Define a function to create a figure displaying seasonal anomalies of a variable
def plot_seasonal_anomaly(var, ax, **kwargs):
    """
    Create a figure displaying seasonal anomalies of a variable.

    Parameters:
    -----------
    var : xarray.DataArray
        The input variable with seasonal data.

    ax : list of matplotlib.Axes
        A list of subplot axes where the seasonal anomalies will be plotted.

    kwargs : keyword arguments
        Additional keyword arguments to customize the plots.

    Returns:
    --------
    hmp : xarray.DataArray
        The result of the plot function.
    """
    crs = ccrs.PlateCarree()

    for i, a in enumerate(ax):
        hmp = (var.isel(season=i) - var.mean('season')).plot(x='lon', transform=crs, ax=a, **kwargs)

    return hmp

# Define a function to create a figure displaying the mean anomaly of a variable and sea ice contours
def plot_mean_anomaly(ax, var, var_lab, units, si, cmap, vmin, vmax, loc1, szn_vmax, szn_cmap, loc2, shrink=0.7, asp=12, fs=15,
                      set_ticks=False, ticks=[], set_ticks2=False, ticks2=[], si_x=15):
    """
    Create a figure displaying the mean anomaly of a variable and sea ice contours.

    Parameters:
    -----------
    ax : list of matplotlib.Axes
        A list of subplot axes where the plots will be displayed.

    var : xarray.DataArray
        The input variable with seasonal data.

    var_lab : str
        The label for the variable.

    units : str
        The units of the variable.

    si : xarray.DataArray
        Sea ice data.

    cmap : str
        The colormap for the plot.

    vmin : float
        The minimum value for the color scale.

    vmax : float
        The maximum value for the color scale.

    szn_vmax : float
        The maximum value for the seasonal anomaly plot.

    szn_cmap : str
        The colormap for the seasonal anomaly plot.

    loc1,loc2 : str
        The location of colorbar for mean plot or seasonal anomaly plot
    
    shrink : float, optional
        The shrink factor for the colorbars.

    asp : int, optional
        The aspect ratio of the colorbars.

    fs : int, optional
        The fontsize for labels and colorbar labels.

    set_ticks : bool, optional
        Whether to set colorbar ticks.

    ticks : list, optional
        Custom colorbar ticks.

    set_ticks2 : bool, optional
        Whether to set colorbar ticks for the seasonal anomaly plot.

    ticks2 : list, optional
        Custom colorbar ticks for the seasonal anomaly plot.

    si_x : int, optional
        Sea ice parameter.

    Returns:
    --------
    None
    """
    crs = ccrs.PlateCarree()

    # Plot the mean
    hmp = var.mean('season').plot(x='lon', transform=crs, ax=ax[0], add_colorbar=False, vmin=vmin, vmax=vmax, cmap=cmap)
    cb = fig.colorbar(hmp, ax=ax[0], shrink=shrink, aspect=asp, location=loc1)
    cb.set_label(label=var_lab + '\n(' + units + ')', fontsize=fs)
    if set_ticks:
        cb.set_ticks(ticks)

    # Plot the anomaly
    hmp = plot_seasonal_anomaly(var, ax[1:5], vmin=-szn_vmax, vmax=szn_vmax, cmap=szn_cmap, add_colorbar=False)
    cb = fig.colorbar(hmp, ax=ax[1:5], shrink=shrink/2, aspect=asp*2, location=loc2, extend='both')
    cb.set_label(label='Anomaly\n(' + units + ')', fontsize=fs)
    if set_ticks2:
        cb.set_ticks(ticks2)

    # Plot sea ice contour
    ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'), nx=20
                       ).plot(ax=ax[0], c='w', transform=crs, lw=1.25)
    for i, a in enumerate(ax[1:5]):
        ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=i), nx=20
                           ).plot(ax=a, c='w', transform=crs, lw=1.25)

In [ ]:
import importlib
importlib.reload(sys.modules['plot_formatting'])
from plot_formatting import *

In [ ]:
crs = ccrs.PlateCarree()

fig = plt.figure(figsize=(8, 14), constrained_layout=True, dpi=600)
gs = gridspec.GridSpec(5, 2, figure=fig,) # height_ratios=[1, 1, 1, 1, 0.8],)

# Create subplots for the first row (excluding the colorbar space)
ax = []

# Create subplots for the second and third rows
for i in range(2):
    for j in range(5):
        ax.append(fig.add_subplot(gs[j, i],projection=ccrs.SouthPolarStereo()))
        ax[-1].set_facecolor('darkgrey')

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)
    
# plot core temp
plot_mean_anomaly(ax[0:5],ds_ww.ww_ct,'Core Temperature',si=si,units='°C',cmap='cmo.thermal',vmin=-2,vmax=2,
                          szn_vmax=0.7,szn_cmap='cmo.curl',loc1='left',loc2='left',)
# plot core sal
plot_mean_anomaly(ax[5:10],ds_ww.ww_sa,'Core Salinity',si=si,units='g/kg',cmap='cmo.haline',vmin=34,vmax=34.6,
                          szn_vmax=0.25,szn_cmap='cmo.delta',loc1='right',loc2='right')
# title formatting
for a in ax:
    a.set_title('')

x,y = 0.97,0.99 #fig.text(x=x,y=y,s=s,transform=a.transAxes,fontsize=20,ha='center')
fig.text(x=x,y=y,s='Mean',transform=ax[0].transAxes,fontsize=20,ha='center')
for a,s in zip(ax[1:6],szn):
    fig.text(x=x,y=y,s=s,transform=a.transAxes,fontsize=20,ha='center')

In [ ]:
crs = ccrs.PlateCarree()

fig = plt.figure(figsize=(8, 14), constrained_layout=True, dpi=600)
gs = gridspec.GridSpec(2, 5, figure=fig,) # height_ratios=[1, 1, 1, 1, 0.8],)

# Create subplots for the first row (excluding the colorbar space)
ax = []

# Create subplots for the second and third rows
for i in range(2):
    for j in range(5):
        ax.append(fig.add_subplot(gs[j, i],projection=ccrs.SouthPolarStereo()))
        ax[-1].set_facecolor('darkgrey')

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)
    
# plot core temp
plot_mean_anomaly(ax[0:5],ds_ww.ww_ct,'Core Temperature',si=si,units='°C',cmap='cmo.thermal',vmin=-2,vmax=2,
                          szn_vmax=0.7,szn_cmap='cmo.curl',loc1='left',loc2='left',)
# plot core sal
plot_mean_anomaly(ax[5:10],ds_ww.ww_sa,'Core Salinity',si=si,units='g/kg',cmap='cmo.haline',vmin=34,vmax=34.6,
                          szn_vmax=0.25,szn_cmap='cmo.delta',loc1='right',loc2='right')
# title formatting
for a in ax:
    a.set_title('')

x,y = 0.97,0.99
fig.text(x=x,y=y,s='Mean',transform=ax[0].transAxes,fontsize=20,ha='center')
for a,s in zip(ax[1:6],szn):
    fig.text(x=x,y=y,s=s,transform=a.transAxes,fontsize=20,ha='center')

In [ ]:
fig.savefig(path+'/figs/04-core_T_S.png',format='png')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(2,5,figsize=[15,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)
    
# plot core temp
plot_mean_anomaly(ax[0:5],ds_ww.ww_ct,'Core Temperature',si=si,units='°C',cmap='cmo.thermal',vmin=-2,vmax=2,
                          szn_vmax=0.7,szn_cmap='cmo.curl')
# plot core sal
plot_mean_anomaly(ax[5:10],ds_ww.ww_sa,'Core Salinity',si=si,units='g/kg',cmap='cmo.haline',vmin=34.15,vmax=34.65,
                          szn_vmax=0.25,szn_cmap='cmo.delta')
# title formatting
ax[0].set_title('Mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
# set all other titles to blank
for a in ax[6:]:
    a.set_title('')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(2,5,figsize=[15,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)
    
# plot core temp
plot_mean_anomaly(ax[0:5],ds_ww.ww_ct,'Core Temperature',si=si,units='°C',cmap='cmo.thermal',vmin=-2,vmax=2,
                          szn_vmax=0.7,szn_cmap='cmo.curl')
# plot core sal
plot_mean_anomaly(ax[5:10],ds_ww.ww_sa,'Core Salinity',si=si,units='g/kg',cmap='cmo.haline',vmin=34.15,vmax=34.65,
                          szn_vmax=0.25,szn_cmap='cmo.delta')
# title formatting
ax[0].set_title('Mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
# set all other titles to blank
for a in ax[6:]:
    a.set_title('')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['ww_ct', 'ww_sa']

print('entire SO')
for v in var:
    stats_var(ds_ww,v)
    print('NEXT 🫡')

print('\n''NORTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat>ice),v)
    print('NEXT 🫡')

print('\n''SOUTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat<=ice),v)
    print('NEXT 🫡')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')
var = ['ww_ct', 'ww_sa']

print('entire SO')
for v in var:
    stats_var(ds_ww,v)
    print('NEXT 🫡')

print('\n''NORTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat>ice),v)
    print('NEXT 🫡')

print('\n''SOUTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat<=ice),v)
    print('NEXT 🫡')

# Fig 3: core depth and thickness

In [ ]:
ds.ww_prof.sum(('lon','lat'),skipna=True)

In [ ]:
crs = ccrs.PlateCarree()

fig = plt.figure(figsize=(8, 14), constrained_layout=True, dpi=600)
gs = gridspec.GridSpec(5, 2, figure=fig,) # height_ratios=[1, 1, 1, 1, 0.8],)

# Create subplots for the first row (excluding the colorbar space)
ax = []

# Create subplots for the second and third rows
for i in range(2):
    for j in range(5):
        ax.append(fig.add_subplot(gs[j, i],projection=ccrs.SouthPolarStereo()))
        ax[-1].set_facecolor('darkgrey')

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)

# plot core depth
plot_mean_anomaly(ax[0:5],ds_ww.ww_cd,'Core Depth',si=si,units='m',cmap='cmo.deep',vmin=0,vmax=175,
                          szn_vmax=50,szn_cmap='cmo.tarn',loc1='left',loc2='left')
# plot core thickness
plot_mean_anomaly(ax[5:10],ds_ww.thcc,'Thickness',si=si,units='m',cmap='Purples',vmin=0,vmax=150,
                          szn_vmax=50,szn_cmap='cmo.balance',loc1='right',loc2='right')

# title formatting
for a in ax:
    a.set_title('')

x,y = 0.97,0.99
fig.text(x=x,y=y,s='Mean',transform=ax[0].transAxes,fontsize=20,ha='center')
for a,s in zip(ax[1:6],szn):
    fig.text(x=x,y=y,s=s,transform=a.transAxes,fontsize=20,ha='center')

In [ ]:
fig.savefig(path+'/figs/05-core_depth_thcc',format='png')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')

var = ['ww_cd','thcc']
print('entire SO')
for v in var:
    stats_var(ds_ww,v)
    print('NEXT 🫡')

print('\n''NORTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat>ice),v)
    print('NEXT 🫡')

print('\n''SOUTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat<=ice),v)
    print('NEXT 🫡')

In [ ]:
ice   = (ds_s.sic - 15).__abs__().idxmin(dim='lat')

var = ['thcc']
print('entire SO')
for v in var:
    stats_var(ds_ww,v)
    print('NEXT 🫡')

print('\n''NORTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat>ice),v)
    print('NEXT 🫡')

print('\n''SOUTH OF SEA ICE')
for v in var:
    stats_var(ds_ww.where(ds_ww.lat<=ice),v)
    print('NEXT 🫡')

# Fig: line plots of mean properties across lat and lon 

### funcs

In [ ]:
def plot_var_lat(ds_ww,var,a,c):
    ds_ww[var].mean(('season','lon')).plot(y='lat',ax=a,c=c)
    (ds_ww[var].mean(('season','lon')) + ds_ww[var].std(('season','lon'))).plot(y='lat',c=c,alpha=0.4,ax=a)
    (ds_ww[var].mean(('season','lon')) - ds_ww[var].std(('season','lon'))).plot(y='lat',c=c,alpha=0.4,ax=a)
    
    a.spines['bottom'].set_color(c)
    a.xaxis.label.set_color(c)
    a.tick_params(axis='x', colors=c)

def plot_var_lon(ds_ww,var,a,c):
    ds_ww[var].mean(('season','lat')).plot(x='lon',ax=a,c=c)
    (ds_ww[var].mean(('season','lat')) + ds_ww[var].std(('season','lat'))).plot(x='lon',c=c,alpha=0.4,ax=a)
    (ds_ww[var].mean(('season','lat')) - ds_ww[var].std(('season','lat'))).plot(x='lon',c=c,alpha=0.4,ax=a)
    
    a.spines['left'].set_color(c)
    a.yaxis.label.set_color(c)
    a.tick_params(axis='y', colors=c)

def set_share_axes(axs, target=None, sharex=False, sharey=False):
    if target is None:
        target = axs.flat[0]
    # Manage share using grouper objects
    for ax in axs.flat:
        if sharex:
            target._shared_axes['x'].join(target, ax)
        if sharey:
            target._shared_axes['y'].join(target, ax)
    # Turn off x tick labels and offset text for all but the bottom row
    if sharex and axs.ndim > 1:
        for ax in axs[:-1,:].flat:
            ax.xaxis.set_tick_params(which='both', labelbottom=False, labeltop=False)
            ax.xaxis.offsetText.set_visible(False)
    # Turn off y tick labels and offset text for all but the left most column
    if sharey and axs.ndim > 1:
        for ax in axs[:,1:].flat:
            ax.yaxis.set_tick_params(which='both', labelleft=False, labelright=False)
            ax.yaxis.offsetText.set_visible(False)

In [ ]:
alphbt = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
          'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

## plots

In [ ]:
import importlib
importlib.reload(sys.modules['plot_formatting'])
from plot_formatting import *

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(14,4))

plot_bathym(ax,bth)

# identify key bathymetry features
add_bathym_vlines(ax)
ax.axes.axvline(50, ls='--',c='grey') # SEIR

In [ ]:
def add_bathym_labels(fig,ax,y=1.1):
    for s,x in zip(['PAR', 'SSA', 'SWIR', 'KP', 'SEIR'],((np.asarray([-145,-30,25,79,145,])+180)%360)/360):
        fig.text(x=x,y=y,s=s,transform=ax.transAxes,ha='center',c='grey',fontsize=15,weight='bold')

In [ ]:
alphbt = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
          'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

fig,ax = plt.subplots(2,2,figsize=(12,6),width_ratios=[4,1],dpi=600)
ax = ax.flatten()

set_share_axes(ax[::2], sharex=True)
set_share_axes(ax[1::2],sharey=True)

c      = ['#e41a1c','#377eb8',]
v      = ['ww_ct','ww_sa',]
titles = ['Core Temperature (°C)','Core Salinity (g/kg)',]
lims   = [(-2,2),(33.9,34.7),]
ticks  = [[-2,-1,0,1,2],np.arange(33.8,34.9,0.2).round(2)]
fs     = 15

for i,(col,var,a,l,t,tks) in enumerate(zip(c,v,ax[::2],lims,titles,ticks)):
    plot_var_lon(ds_ww,var,a,col)
    a.set_xlabel('')
    a.set_xticklabels([])
    a.set_yticks(tks)
    a.set_ylim(l)
    a.set_ylabel(t,fontsize=fs)
    a.text(x=0.01,y=0.92,s="(" + alphbt[::2][i] + ")",transform=a.transAxes)
    add_bathym_vlines(a)
add_bathym_labels(fig,ax[-2])
#ax[0].set_ylabel('Core''\n''Temperature (°C)',fontsize=fs)


for i,(col,var,a,l,t,tks) in enumerate(zip(c,v,ax[1::2],lims,titles,ticks)):
    plot_var_lat(ds_ww,var,a,col)
    a.axhline(wrap_smth_var((si.sic - 15).__abs__().idxmin(dim='lat').mean('season'),nx=20).mean('lon'),
              ls = '--',c='k')
    
    a.yaxis.tick_right()
    a.yaxis.set_label_position("right")
    a.set_ylabel('Latitude (°N)',fontsize=fs)
    a.set_xlabel(t,fontsize=fs)
    a.set_xticks(tks)
    a.set_xticklabels(tks)
    a.set_xlim(l)
    
    a.text(x=0.02,y=0.92,s="(" + alphbt[1::2][i] + ")",transform=a.transAxes)

ax[-2].set_xlabel('Longitude (°E)',fontsize=fs)
ax[-2].set_xlim(-180,180)
ax[-2].set_xticklabels(np.arange(-200,185,50))

fig.tight_layout()

In [ ]:
alphbt = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
          'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

fig,ax = plt.subplots(2,2,figsize=(12,6),width_ratios=[4,1],dpi=600)
ax = ax.flatten()

set_share_axes(ax[::2], sharex=True)
set_share_axes(ax[1::2],sharey=True)

# define vars to plot and their associated props
c      = ['#4daf4a','#984ea3']
v      = ['ww_cd','thcc']
titles = ['Core Depth (m)','Thickness (m)']
lims   = [(0,200),(20,180)]
fs     = 15

for i,(col,var,a,l,t) in enumerate(zip(c,v,ax[::2],lims,titles)):
    plot_var_lon(ds_ww,var,a,col)
    a.set_xlabel('')
    a.set_xticklabels([])
    a.set_ylim(l)
    a.set_ylabel(t,fontsize=fs)
    add_bathym_vlines(a)
    a.text(x=0.01,y=0.92,s="(" + alphbt[::2][i] + ")",transform=a.transAxes)
add_bathym_labels(fig,ax[-2])

for i,(col,var,a,l,t) in enumerate(zip(c,v,ax[1::2],lims,titles)):
    plot_var_lat(ds_ww,var,a,col)
    a.axhline(wrap_smth_var((si.sic - 15).__abs__().idxmin(dim='lat').mean('season'),nx=20).mean('lon'),
              ls = '--',c='k')
    
    a.yaxis.tick_right()
    a.yaxis.set_label_position("right")
    a.set_xlabel(t,fontsize=fs)
    a.set_ylabel('Latitude (°N)',fontsize=fs)
    a.set_xlim(l)
    
    a.text(x=0.02,y=0.92,s="(" + alphbt[1::2][i] + ")",transform=a.transAxes)
    
ax[-2].set_xlabel('Longitude (°E)',fontsize=fs)
ax[-2].set_xlim(-180,180)
ax[-2].set_xticklabels(np.arange(-200,185,50))

fig.tight_layout()

In [ ]:
alphbt = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 
          'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

fig,ax = plt.subplots(5,2,figsize=(14,12.5),width_ratios=[4,1],dpi=600)
ax = ax.flatten()

set_share_axes(ax[::2],sharex=True)

c      = ['#e41a1c','#377eb8','#984ea3','#4daf4a','#ff7f00']
v      = ['ww_ct','ww_sa','sig_c','ww_cd','thcc']
titles = ['Core Temperature (°C)','Core Salinity (g/kg)','Core Density (kg/m$^{-3}$','Core Depth (m)','Thickness (m)']
lims   = [(-2,2),(33.8,34.8),(27,28),(0,200),(40,180)]
#ticks  = [[-2,0,2]]

for i,(col,var,a,l,t) in enumerate(zip(c,v,ax[::2],lims,titles)):
    plot_var_lon(ds_ww,var,a,col)
    a.set_xlabel('')
    a.set_xticklabels([])
    a.set_ylim(l)
    a.set_ylabel(t)

    # identify key bathymetry features
    add_bathym_vlines(a)

    a.text(x=0.01,y=0.87,s="(" + alphbt[::2][i] + ")",transform=a.transAxes)

#ax[0].set_ylabel('')
#ax[0].set_ylabel('Core\nTemperature (°C)')

for i,(col,var,a,l,t) in enumerate(zip(c,v,ax[1::2],lims,titles)):
    plot_var_lat(ds_ww,var,a,col)
    a.axhline(wrap_smth_var((si.sic - 15).__abs__().idxmin(dim='lat').mean('season'),nx=20).mean('lon'),
              ls = '--',c='k')
    
    a.yaxis.tick_right()
    a.yaxis.set_label_position("right")
    a.set_xlabel(t)
    a.set_ylabel('Latitude (°N)')
    a.set_xlim(l)
    
    a.text(x=0.02,y=0.87,s="(" + alphbt[1::2][i] + ")",transform=a.transAxes)
    
ax[-2].set_xlabel('Longitude (°E)')
ax[-2].set_xlim(-180,180)
ax[-2].set_xticklabels(np.arange(-200,185,50))

fig.tight_layout()

## less usfeul plots

In [ ]:
fig,ax = plt.subplots(3,2,figsize=(10,12),sharex=False,sharey=True, dpi=600)
ax = ax.flatten()

c = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
v = ['ww_ct','ww_sa','ww_cd','thcc','sig_c']
for col,var,a in zip(c,v,ax):
    plot_var_lat(ds_ww,var,a,col)
    a.set_ylabel('')

ax[0].set_xlim(-2,2)
ax[0].set_xlabel('Core Temperature (°C)')

ax[1].set_xlim(33.8,34.8)
ax[1].set_xlabel('Core Salinity (°C)')

ax[2].set_xlabel('Core Depth (m)')

ax[3].set_xlabel('Thickness (m)')

ax[4].set_xlabel('Core Density (kg/m3)')

for a in ax[::2]:
    a.set_ylabel('Latitude (°N)')

fig.tight_layout()


fig.delaxes(ax[-1])

In [ ]:
fig,ax = plt.subplots(2,2,figsize=(10,10),sharex=False,sharey=True, dpi=600)
ax = ax.flatten()

c = ['#e41a1c','#377eb8','#4daf4a','#984ea3',]#'#ff7f00']
v = ['ww_ct','ww_sa','ww_cd','thcc',] #'sig_c']
for col,var,a in zip(c,v,ax):
    plot_var_lat(ds_ww,var,a,col)
    a.set_ylabel('')

ax[0].set_xlim(-2,2)
ax[0].set_xlabel('Core Temperature (°C)')

ax[1].set_xlim(33.8,34.8)
ax[1].set_xlabel('Core Salinity (°C)')

ax[2].set_xlabel('Core Depth (m)')
ax[2].set_xlim(25,200)

ax[3].set_xlabel('Thickness (m)')
ax[3].set_xlim(40,180)

#ax[4].set_xlabel('Core Density (kg/m3)')

for a in ax[::2]:
    a.set_ylabel('Latitude (°N)')

fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(5,1,figsize=(14,10),sharex=True,sharey=False, dpi=600)
ax = ax.flatten()

c = ['#e41a1c','#377eb8','#4daf4a','#984ea3','#ff7f00']
v = ['ww_ct','ww_sa','ww_cd','thcc','sig_c']
for col,var,a in zip(c,v,ax):
    plot_var_lon(ds_ww,var,a,col)
    a.set_xlabel('')

ax[0].set_ylim(-2,2)
ax[0].set_ylabel('Core Temperature (°C)')

ax[1].set_ylim(33.8,34.8)
ax[1].set_ylabel('Core Salinity (°C)')

ax[2].set_ylabel('Core Depth (m)')

ax[3].set_ylabel('Thickness (m)')

ax[4].set_ylabel('Core Density (kg/m3)')

ax[-1].set_xlabel('Longitude (°E)')

# check some density-related stuff

In [ ]:
boxplot(ds_s.sig.where(ds.ww_msk==1).diff('season'))

In [ ]:
ds.sig.where(ds.ww_msk==1).mean('pres',skipna=True).where(~np.logical_and(ds.ww_type==0,ds.ww_type==1)
        ).plot(row='season',x='lon',figsize=(14,10),vmin=27,vmax=27.8,cmap='cmo.dense')

In [ ]:
ds_ww['sig'] = nan_interp(wrap_smth_var(ds.sig.where(ds.ww_msk==1).mean('pres',skipna=True
                        ).where(~np.logical_and(ds.ww_type==0,ds.ww_type==1))))

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(1,5,figsize=[15,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
circular_plot_fomatting(fig,ax,bathym=False,sea_ice=False)
    
for a in ax[::5]:
    ice = (si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'
        ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)
x = np.concatenate([np.arange(1,5,1)]) #,np.arange(6,10,1)])
for i,a in enumerate(ax[x]):
    ice = (si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=int(i%4)
        ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)

# plot core depth (pressure)
hmp = ds_ww.sig.mean('season').plot(
                        x='lon',transform=crs,vmin=27.05,vmax=27.65,cmap='cmo.dense',
                        ax=ax[0],add_colorbar=False)
ax[0].set_title('mean')
cb = fig.colorbar(hmp,ax=ax[0],shrink=0.5,aspect=asp,location='left')
cb.set_label(label='density, $\sigma_0$ (kg/m3)',fontsize=fs)

# seasonal anomaly from mean
for i,a in enumerate(ax[1:5]):
    hmp = (-ds_ww.sig.mean('season') + ds_ww.sig.isel(season=i)).plot(
                        x='lon',transform=crs,vmin=-0.15,vmax=0.15,cmap='cmo.balance',#levels=9,
                        ax=ax[1+(int(i+1)%4)],add_colorbar=False)
    a.set_title(szn[i%4]) #[:3])
cb = fig.colorbar(hmp,ax=ax,shrink=0.5,aspect=asp,)
cb.set_label(label='Anomaly (kg/m3)',fontsize=fs)

ax[1].set_title(szn[0])

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(4,5,figsize=[15,9.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
circular_plot_fomatting(fig,ax,bathym=False,sea_ice=False)

lvls = [25.6,27.3,27.6,27.9,28.7] # density grouping according to Pellichero et al 2018

for j in range(4):
    tmp = ds_ww.sig.where(np.logical_and(ds_ww.sig>=lvls[j],ds_ww.sig<lvls[j+1]))
    
    # plot annual mean for density group                      
    hmp1 = tmp.mean('season').plot(
                        x='lon',transform=crs,vmin=27.05,vmax=27.65,cmap='cmo.dense',
                        ax=ax[j*5],add_colorbar=False)
    a.set_title('')
    
    fig.text(-0.065,1-(1/5*(j+1) + 0.05),s=str(lvls[j])+r' ≤ $\sigma$ < '+str(lvls[j+1]),fontsize=12,)
    
    # plot seasonal anom for density group
    ax2 = ax[1+j*(5):(j+1)*(4+1)]
    for i,a in enumerate(ax2):
        ice = (si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=int(i%4)
            ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)

        hmp2 = (-tmp.mean('season') + tmp.isel(season=i)).plot(
                        x='lon',transform=crs,vmin=-0.15,vmax=0.15,cmap='cmo.balance',#levels=9,
                        ax=ax2[(int(i+1)%4)],add_colorbar=False)
        a.set_title('')
        
        if j==0:
            a.set_title(szn[i%4]) #[:3])


ax[0].set_title('mean')

cb = fig.colorbar(hmp1,ax=ax,shrink=0.5,aspect=asp*2,location='left')
cb.set_label(label='density, $\sigma_0$ (kg/m3)',fontsize=fs)
                          
cb = fig.colorbar(hmp2,ax=ax,shrink=0.5,aspect=asp*2,)
cb.set_label(label='Anomaly (kg/m3)',fontsize=fs)

for a in ax[1::5]:
    a.set_title('')
ax[1].set_title(szn[0])

fig.show()

# aiding figs

In [ ]:
boxplotent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(1,5,figsize=[17.5,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
for i,a in enumerate(ax):
    circular_boundary(a)
    a.set_extent(extent,crs=crs)
    a.add_feature(cfeature.LAND,zorder=9,facecolor="#ededed",)
    a.coastlines(zorder=10,color="k") # shows continental coastline
    
    # add front contours and bathym
    frnt = ds.adt.mean('season').plot.contour(x='lon',transform=crs,ax=a,levels=[-0.58,-0.1],colors='k',
                                       linewidths=1.5,linestyles=['-'],labels='PF & SAF')
    b   = bth.elevation.plot.contour(x='lon',transform=crs,ax=a,levels=[-1e3,-2e3,-3e3],
                                     colors='#4c4c4c',linestyles=['-'],linewidths=[0.7],alphas=[0.9])
    ice = (si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'
        ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)
# add legend of sea ice, fronts and bathym
#fig.legend(handles=[frnt.collections[0],
 #                   ice[0],
  #                  b.collections[0]],
   #        labels=("PF,SAF",
    #               f"sea ice\nconcentration ({si_x}%)",
     #              "1,2,3km\nisobaths"),
      #     loc=(0.9-0.025,0.9+0.01),facecolor='#c3c3c3')

ds_ww.ww_cp.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[0],vmin=-50,vmax=50,cmap='cmo.balance',add_colorbar=False)
ax[0].set_title('core depth')

hmp = ds_ww.thcc.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[1],vmin=-50,vmax=50,cmap='cmo.balance',add_colorbar=False)
ax[1].set_title('thickness')
cb = fig.colorbar(hmp,ax=ax[:2],shrink=0.7,aspect=asp*2,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.ww_ct.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[2],vmin=-0.5,vmax=0.5,cmap='cmo.curl',add_colorbar=False)
ax[2].set_title('core temp')
cb = fig.colorbar(hmp,ax=ax[2],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='°C',fontsize=fs)

hmp = ds_ww.ww_sa.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[3],vmin=-0.3,vmax=0.3,cmap='cmo.delta',add_colorbar=False)
ax[3].set_title('core asal')
cb = fig.colorbar(hmp,ax=ax[3],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='g/kg',fontsize=fs)

hmp = (ds_ww.ww_n2_int.mean('season').diff('lat')*1e3).plot(
            x='lon',transform=crs,ax=ax[4],vmin=-0.3,vmax=0.3,cmap='cmo.diff',add_colorbar=False)
ax[4].set_title('WW integrated n2')
cb = fig.colorbar(hmp,ax=ax[4],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label=r'$\times 10^{-3}s^{-1}$',fontsize=fs)

fig.suptitle(r'$\frac{d\,\overline{var}}{d\,lat}$')

In [ ]:
boxplotent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(1,5,figsize=[17.5,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
for i,a in enumerate(ax):
    circular_boundary(a)
    a.set_extent(extent,crs=crs)
    a.add_feature(cfeature.LAND,zorder=9,facecolor="#ededed",)
    a.coastlines(zorder=10,color="k") # shows continental coastline
    
    # add front contours and bathym
    frnt = ds.adt.mean('season').plot.contour(x='lon',transform=crs,ax=a,levels=[-0.58,-0.1],colors='k',
                                       linewidths=1.5,linestyles=['-'],labels='PF & SAF')
    b   = bth.elevation.plot.contour(x='lon',transform=crs,ax=a,levels=[-1e3,-2e3,-3e3],
                                     colors='#4c4c4c',linestyles=['-'],linewidths=[0.7],alphas=[0.9])
    ice = (si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'
        ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)
# add legend of sea ice, fronts and bathym
#fig.legend(handles=[frnt.collections[0],
 #                   ice[0],
  #                  b.collections[0]],
   #        labels=("PF,SAF",
    #               f"sea ice\nconcentration ({si_x}%)",
     #              "1,2,3km\nisobaths"),
      #     loc=(0.9-0.025,0.9+0.01),facecolor='#c3c3c3')

ds_ww.ww_cp.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[0],vmin=-50,vmax=50,cmap='cmo.balance',add_colorbar=False)
ax[0].set_title('core depth')

hmp = ds_ww.thcc.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[1],vmin=-50,vmax=50,cmap='cmo.balance',add_colorbar=False)
ax[1].set_title('thickness')
cb = fig.colorbar(hmp,ax=ax[:2],shrink=0.7,aspect=asp*2,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.ww_ct.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[2],vmin=-0.5,vmax=0.5,cmap='cmo.curl',add_colorbar=False)
ax[2].set_title('core temp')
cb = fig.colorbar(hmp,ax=ax[2],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='°C',fontsize=fs)

hmp = ds_ww.ww_sa.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[3],vmin=-0.3,vmax=0.3,cmap='cmo.delta',add_colorbar=False)
ax[3].set_title('core asal')
cb = fig.colorbar(hmp,ax=ax[3],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='g/kg',fontsize=fs)

hmp = (ds_ww.ww_n2.mean('season').diff('lat')*1e5).plot(
            x='lon',transform=crs,ax=ax[4],vmin=-0.75,vmax=0.75,cmap='cmo.diff',add_colorbar=False)
ax[4].set_title('WW mean n2')
cb = fig.colorbar(hmp,ax=ax[4],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label=r'$\times 10^{-5}s^{-1}$',fontsize=fs)

fig.suptitle(r'$\frac{d\,\overline{var}}{d\,lat}$')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(1,5,figsize=[17.5,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
for i,a in enumerate(ax):
    circular_boundary(a)
    a.set_extent(extent,crs=crs)
    a.add_feature(cfeature.LAND,zorder=9,facecolor="#ededed",)
    a.coastlines(zorder=10,color="k") # shows continental coastline
    
    # add front contours and bathym
    frnt = ds.adt.mean('season').plot.contour(x='lon',transform=crs,ax=a,levels=[-0.58,-0.1],colors='k',
                                       linewidths=1.5,linestyles=['-'],labels='PF & SAF')
    b   = bth.elevation.plot.contour(x='lon',transform=crs,ax=a,levels=[-1e3,-2e3,-3e3],
                                     colors='#4c4c4c',linestyles=['-'],linewidths=[0.7],alphas=[0.9])
    ice = (si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'
        ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)
# add legend of sea ice, fronts and bathym
#fig.legend(handles=[frnt.collections[0],
 #                   ice[0],
  #                  b.collections[0]],
   #        labels=("PF,SAF",
    #               f"sea ice\nconcentration ({si_x}%)",
     #              "1,2,3km\nisobaths"),
      #     loc=(0.9-0.025,0.9+0.01),facecolor='#c3c3c3')

ds_ww.ww_cp.mean('season').diff('lon').plot(x='lon',transform=crs,ax=ax[0],vmin=-10,vmax=10,cmap='cmo.balance',add_colorbar=False)
ax[0].set_title('core depth')

hmp = ds_ww.thcc.mean('season').diff('lon').plot(x='lon',transform=crs,ax=ax[1],vmin=-10,vmax=10,cmap='cmo.balance',add_colorbar=False)
ax[1].set_title('thickness')
cb = fig.colorbar(hmp,ax=ax[:2],shrink=0.7,aspect=asp*2,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.ww_ct.mean('season').diff('lon').plot(x='lon',transform=crs,ax=ax[2],vmin=-0.2,vmax=0.2,cmap='cmo.curl',add_colorbar=False)
ax[2].set_title('core temp')
cb = fig.colorbar(hmp,ax=ax[2],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='°C',fontsize=fs)

hmp = ds_ww.ww_sa.mean('season').diff('lon').plot(x='lon',transform=crs,ax=ax[3],vmin=-0.04,vmax=0.04,cmap='cmo.delta',add_colorbar=False)
ax[3].set_title('core asal')
cb = fig.colorbar(hmp,ax=ax[3],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='g/kg',fontsize=fs)

hmp = (ds_ww.ww_n2_int.mean('season').diff('lon')*1e3).plot(
            x='lon',transform=crs,ax=ax[4],vmin=-0.2,vmax=0.2,cmap='cmo.diff',add_colorbar=False)
ax[4].set_title('WW mean n2')
cb = fig.colorbar(hmp,ax=ax[4],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label=r'$\times 10^{-3}s^{-1}$',fontsize=fs)

fig.suptitle(r'$\frac{d\,\overline{var}}{d\,lon}$')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(1,5,figsize=[17.5,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
for i,a in enumerate(ax):
    circular_boundary(a)
    a.set_extent(extent,crs=crs)
    a.add_feature(cfeature.LAND,zorder=9,facecolor="#ededed",)
    a.coastlines(zorder=10,color="k") # shows continental coastline
    
    # add front contours and bathym
    frnt = ds.adt.mean('season').plot.contour(x='lon',transform=crs,ax=a,levels=[-0.58,-0.1],colors='k',
                                       linewidths=1.5,linestyles=['-'],labels='PF & SAF')
#    b   = bth.elevation.plot.contour(x='lon',transform=crs,ax=a,levels=[-1e3,-2e3,-3e3],
 #                                    colors='#4c4c4c',linestyles=['-'],linewidths=[0.7],alphas=[0.9])
    ice = (si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'
        ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)
# add legend of sea ice, fronts and bathym
#fig.legend(handles=[frnt.collections[0],
 #                   ice[0],
  #                  b.collections[0]],
   #        labels=("PF,SAF",
    #               f"sea ice\nconcentration ({si_x}%)",
     #              "1,2,3km\nisobaths"),
      #     loc=(0.9-0.025,0.9+0.01),facecolor='#c3c3c3')

# add basic kwargs for circump plots
kwargs = dict(x='lon',transform=crs,vmin=0,add_colorbar=False,) 
ds_ww.ww_cp.std('season').plot(**kwargs,ax=ax[0],vmax=50,cmap='cmo.amp',)
ax[0].set_title('core depth')

hmp = ds_ww.thcc.std('season').plot(**kwargs,ax=ax[1],vmax=50,cmap='cmo.amp',)
ax[1].set_title('thickness')
cb = fig.colorbar(hmp,ax=ax[:2],shrink=0.7,aspect=asp*2,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.ww_ct.std('season').plot(**kwargs,ax=ax[2],vmax=0.5,cmap='Greens',)
ax[2].set_title('core temp')
cb = fig.colorbar(hmp,ax=ax[2],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='°C',fontsize=fs)

hmp = ds_ww.ww_sa.std('season').plot(**kwargs,ax=ax[3],vmax=0.3,cmap='BuGn',)
ax[3].set_title('core asal')
cb = fig.colorbar(hmp,ax=ax[3],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='g/kg',fontsize=fs)

hmp = (ds_ww.ww_n2_int.std('season')*1e3).plot(**kwargs,ax=ax[4],vmax=1,cmap='Blues',)
ax[4].set_title('WW integrated n2')
cb = fig.colorbar(hmp,ax=ax[4],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label=r'$\times 10^{-3}s^{-1}$',fontsize=fs)

fig.text(-0.065,0.5,s=r'$\sigma(var)$',fontsize=25,)

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(1,5,figsize=[17.5,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
circular_plot_fomatting(fig,ax,bathym=True,sea_ice=False)

kwargs = dict(x='lon',transform=crs,vmin=0,vmax=125,cmap='cmo.deep',add_colorbar=False,) 
hmp = ds_s.mlp.mean('season').plot(**kwargs,ax=ax[0])
ax[0].set_title('mld')
cb = fig.colorbar(hmp,ax=ax[0],shrink=0.7,aspect=asp,location='left')


# add basic kwargs for circump plots
kwargs = dict(x='lon',transform=crs,vmin=-100,vmax=100,cmap='cmo.balance',add_colorbar=False,) 
circular_plot_fomatting(fig,ax[1:],bathym=False,sea_ice=True)
for i,a in enumerate(ax[1:]):
    hmp = (ds_s.mlp.isel(season=i) - ds_s.mlp.mean('season')).plot(**kwargs,ax=a)
    a.set_title(szn[i])
cb = fig.colorbar(hmp,ax=ax[1:],shrink=0.7,aspect=asp,location='right')


In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(1,4,figsize=[17.5,5.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
circular_plot_fomatting(fig,ax,bathym=True,sea_ice=True)

# add basic kwargs for circump plots
kwargs = dict(x='lon',transform=crs,vmin=0,vmax=150,cmap='cmo.deep',add_colorbar=False,)
for i,a in enumerate(ax):
    hmp = ds_s.mlp.isel(season=i).plot(**kwargs,ax=a)
    a.set_title(szn[i])
cb = fig.colorbar(hmp,ax=ax,shrink=0.7,aspect=asp,location='right')

fig.show()

# Supplementary

In [ ]:
def plot_mean_seasonal_change(ax,var,var_lab,units,si,cmap,vmin,vmax,szn_vmax,szn_cmap,shrink=0.7,asp=12,fs=20,
                              set_ticks=False,ticks=[],set_ticks2=False,ticks2=[],xi_x=15):
    crs = ccrs.PlateCarree()
    
    # plot mean
    hmp = var.mean('season').plot(
                            x='lon',transform=crs,ax=ax[0],add_colorbar=False,vmin=vmin,vmax=vmax,cmap=cmap,)
    cb = fig.colorbar(hmp,ax=ax[0],shrink=shrink,aspect=asp,location='left')
    cb.set_label(label=var_lab+'\n('+units+')',fontsize=fs)
    if set_ticks:
        cb.set_ticks(ticks)

    # seasonal change
    hmp = plot_seasonal_diff(var,ax[1:5],vmin=-szn_vmax,vmax=szn_vmax,cmap=szn_cmap,add_colorbar=False,)
    cb = fig.colorbar(hmp,ax=ax[1:5],shrink=shrink,aspect=asp,)
    cb.set_label(label='Seasonal\nChange ('+units+')',fontsize=fs)
    if set_ticks2:
        cb.set_ticks(ticks2)
        
    # plot sea ice contour
    ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'),nx=20
                       ).plot(ax=ax[0],c='w',transform=crs,lw=1.25)
    for i,a in enumerate(ax[1:5]):
        ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=i),nx=20
                           ).plot(ax=a,c='w',transform=crs,lw=1.25)

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(5,5,figsize=[15,13],constrained_layout=True,#dpi=600,
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour
vmax = 60

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)
    
#for a in ax[::5]:
 #   ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'),nx=20
  #                     ).plot(ax=a,c='w',transform=crs,lw=1.25)
#x = np.concatenate([np.arange(1,5,1),np.arange(6,10,1)])
#for i,a in enumerate(ax[x]):
 #   ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=int(i%4)),nx=20
  #                     ).plot(ax=a,c='w',transform=crs,lw=1.25)

# plot core temp
plot_mean_seasonal_change(ax[0:5],ds_ww.ww_ct,'Core Temperature',si=si,units='°C',cmap='cmo.thermal',vmin=-2,vmax=2,
                          szn_vmax=0.9,szn_cmap='cmo.curl')

# plot core sal
plot_mean_seasonal_change(ax[5:10],ds_ww.ww_sa,'Core Salinity',si=si,units='g/kg',cmap='cmo.haline',vmin=34.0,vmax=34.6,
                          szn_vmax=0.3,szn_cmap='cmo.delta')

# plot core depth
plot_mean_seasonal_change(ax[10:15],ds_ww.ww_cd,'Core Depth',si=si,units='m',cmap='cmo.deep',vmin=0,vmax=200,
                          szn_vmax=80,szn_cmap='cmo.tarn')

# plot core thickness
plot_mean_seasonal_change(ax[15:20],ds_ww.thcc,'Thickness',si=si,units='m',cmap='Purples',vmin=0,vmax=150,
                          szn_vmax=60,szn_cmap='cmo.balance')

# plot core density
plot_mean_seasonal_change(ax[20:25],ds_ww.sig_c,'Core Density',si=si,units=r'kg/m$^{-3}$',cmap='cmo.dense',
                          vmin=27.1,vmax=27.7,szn_vmax=0.15,szn_cmap='cmo.diff')

# title formatting
ax[0].set_title('Mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
for a in ax[6:]:
    a.set_title('')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(4,5,figsize=[15,10.5],constrained_layout=True,dpi=600,
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour
vmax = 60

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)
    
#for a in ax[::5]:
 #   ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'),nx=20
  #                     ).plot(ax=a,c='w',transform=crs,lw=1.25)
#x = np.concatenate([np.arange(1,5,1),np.arange(6,10,1)])
#for i,a in enumerate(ax[x]):
 #   ice = wrap_smth_var((si.sic - si_x).__abs__().idxmin(dim='lat').isel(season=int(i%4)),nx=20
  #                     ).plot(ax=a,c='w',transform=crs,lw=1.25)

# plot core temp
plot_mean_seasonal_change(ax[0:5],ds_ww.ww_ct,'Core Temperature',si=si,units='°C',cmap='cmo.thermal',vmin=-2,vmax=2,
                          szn_vmax=0.9,szn_cmap='cmo.curl')

# plot core sal
plot_mean_seasonal_change(ax[5:10],ds_ww.ww_sa,'Core Salinity',si=si,units='g/kg',cmap='cmo.haline',vmin=34.0,vmax=34.6,
                          szn_vmax=0.3,szn_cmap='cmo.delta')

# plot core depth
plot_mean_seasonal_change(ax[10:15],ds_ww.ww_cd,'Core Depth',si=si,units='m',cmap='cmo.deep',vmin=0,vmax=200,
                          szn_vmax=80,szn_cmap='cmo.tarn')

# plot core thickness
plot_mean_seasonal_change(ax[15:20],ds_ww.thcc,'Thickness',si=si,units='m',cmap='Purples',vmin=0,vmax=150,
                          szn_vmax=60,szn_cmap='cmo.balance')

# plot core density
#plot_mean_seasonal_change(ax[20:25],ds_ww.sig_c,'Core Density',si=si,units=r'kg/m$^{-3}$',cmap='cmo.dense',
#                          vmin=27.1,vmax=27.7,szn_vmax=0.15,szn_cmap='cmo.diff')

# title formatting
ax[0].set_title('Mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
for a in ax[6:]:
    a.set_title('')

In [ ]:
ds_ww

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(5,5,figsize=[15,12.5],constrained_layout=True,dpi=300,
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)

plot_mean_seasonal_change(ax[0:5],ds_ww.mld*-1,'MLD',si=si,units='m',cmap='cmo.solar',
                          vmin=0,vmax=130,szn_vmax=70,szn_cmap='cmo.curl')
plot_mean_seasonal_change(ax[5:10],ds_ww.up_bd,'Upper Boundary',si=si,units='m',cmap='cmo.dense',vmin=0,vmax=75,
                          szn_vmax=100,szn_cmap='cmo.diff')
plot_mean_seasonal_change(ax[10:15],ds_ww.lw_bd,'Lower Boundary',si=si,units='m',cmap='cmo.amp',vmin=0,vmax=250,
                          szn_vmax=100,szn_cmap='cmo.diff')
# plot core depth
plot_mean_seasonal_change(ax[15:20],ds_ww.ww_cd,'Core Depth',si=si,units='m',cmap='cmo.deep',vmin=0,vmax=150,
                          szn_vmax=75,szn_cmap='cmo.tarn')
# plot core thickness
plot_mean_seasonal_change(ax[20:25],ds_ww.thcc,'Thickness',si=si,units='m',cmap='Purples',vmin=0,vmax=150,
                          szn_vmax=60,szn_cmap='cmo.balance')
# title formatting
ax[0].set_title('mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
# set all other titles to blank
for a in ax[np.arange(5,25,1)]:
    a.set_title('')

In [ ]:
fig,ax = plt.subplots(3,5,figsize=[15,7.5],constrained_layout=True,dpi=300,
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)

plot_mean_anomaly(fig,ax[0:5],ds_ww.mld,'MLD',units='dbar',si=si,cmap='cmo.amp',vmin=0,vmax=150,
                          szn_vmax=60,szn_cmap='cmo.diff')
plot_mean_anomaly(fig,ax[5:10],ds_ww.up_bd,'Upper\nBoundary',units='dbar',si=si,cmap='cmo.dense',vmin=0,vmax=100,
                          szn_vmax=60,szn_cmap='cmo.curl')
plot_mean_anomaly(fig,ax[10:15],ds_ww.lw_bd,'Lower\nBoundary',units='dbar',si=si,cmap='cmo.deep',vmin=0,vmax=225,
                          szn_vmax=60,szn_cmap='cmo.curl')
# title formatting
ax[0].set_title('mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
# set all other titles to blank
for a in ax[np.arange(5,15,1)]:
    a.set_title('')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(3,5,figsize=[15,2.5*3 + 0.5],constrained_layout=True,dpi=600,
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)

# plot core temp
plot_mean_seasonal_change(ax[0:5],(ds_ww.mld*-1),'MLD',si=si,units='m',cmap='cmo.amp',vmin=0,vmax=150,
                          szn_vmax=60,szn_cmap='cmo.diff')

# plot core sal
plot_mean_seasonal_change(ax[5:10],ds_ww.up_bd,'Upper Boundary',si=si,units='m',cmap='cmo.dense',vmin=0,vmax=100,
                          szn_vmax=60,szn_cmap='cmo.curl')

# plot core depth
plot_mean_seasonal_change(ax[10:15],ds_ww.lw_bd,'Lower Boundary',si=si,units='m',cmap='cmo.deep',vmin=0,vmax=200,
                          szn_vmax=60,szn_cmap='cmo.curl')


# title formatting
ax[0].set_title('Mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
for a in ax[6:]:
    a.set_title('')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(3,5,figsize=[15,2.5*3 + 0.5],constrained_layout=True,dpi=600,
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)

# plot mld
plot_mean_anomaly(ax[0:5],ds_ww.mld*-1,'MLD',si=si,units='m',cmap='cmo.amp',vmin=0,vmax=150,
                          szn_vmax=60,szn_cmap='cmo.diff',)

# plot upper boundary
plot_mean_anomaly(ax[5:10],ds_ww.up_bd,'Upper Boundary',si=si,units='m',cmap='cmo.dense',vmin=0,vmax=100,
                          szn_vmax=60,szn_cmap='cmo.curl')

# plot lower boundary
plot_mean_anomaly(ax[10:15],ds_ww.lw_bd,'Lower Boundary',si=si,units='m',cmap='cmo.deep',vmin=0,vmax=200,
                          szn_vmax=60,szn_cmap='cmo.curl')


# title formatting
ax[0].set_title('Mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
for a in ax[6:]:
    a.set_title('')

In [ ]:
fig,ax = plt.subplots(3,5,figsize=[15,7.5],constrained_layout=True,dpi=300,
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax=ax.flatten()

# run for all axes
circular_plot_fomatting(fig,ax,ds,bth=bth,sea_ice=False)

plot_mean_seasonal_change(ax[0:5],ds_ww.mlp,'MLD',units='dbar',cmap='cmo.amp',vmin=0,vmax=120,
                          szn_vmax=60,szn_cmap='cmo.diff',set_ticks=True,ticks=np.arange(0,160,40))
plot_mean_seasonal_change(ax[5:10],ds_ww.up_bd,'Upper\nBoundary',units='dbar',cmap='cmo.dense',vmin=0,vmax=100,
                          szn_vmax=75,szn_cmap='cmo.curl')
plot_mean_seasonal_change(ax[10:15],ds_ww.lw_bd,'Lower\nBoundary',units='dbar',cmap='cmo.deep',vmin=0,vmax=250,
                          szn_vmax=75,szn_cmap='cmo.curl')
# title formatting
ax[0].set_title('mean')
for i,a in enumerate(ax[1:5]):
    a.set_title(szn[i])
# set all other titles to blank
for a in ax[np.arange(5,15,1)]:
    a.set_title('')

In [ ]:
extent = [180,-180,-90,-45]
crs = ccrs.PlateCarree()

fig,ax = plt.subplots(3,5,figsize=[17.5,15.5],constrained_layout=True,dpi=500,#sharex='col',sharey='row',
                      subplot_kw={'projection':ccrs.SouthPolarStereo(),'facecolor':'darkgrey'})
ax1 = ax.flatten()

fs = 20   # fontsize
asp = 12  # aspect for colorbars
si_x = 15 # sea ice concentration contour

# run for all axes
circular_plot_fomatting(fig,ax1,ds,bth=bth,sea_ice=False)
for i,a in enumerate(ax1):
    ice = (si.sic - si_x).__abs__().idxmin(dim='lat').mean('season'
        ).rolling({'lon':20},min_periods=1).median(skipna=True).plot(ax=a,c='w',transform=crs,lw=1.25)

# define ax for first row
ax = ax1[:5]
hmp = ds_ww.ww_cp.mean('season').plot(x='lon',transform=crs,ax=ax[0],vmin=0,vmax=200,cmap='cmo.deep',add_colorbar=False)
ax[0].set_title('core depth')
cb = fig.colorbar(hmp,ax=ax[0],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.thcc.mean('season').plot(x='lon',transform=crs,ax=ax[1],vmin=0,vmax=200,cmap='Purples',add_colorbar=False)
ax[1].set_title('thickness')
cb = fig.colorbar(hmp,ax=ax[1],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.ww_ct.mean('season').plot(x='lon',transform=crs,ax=ax[2],vmin=-2,vmax=2,cmap='cmo.thermal',add_colorbar=False)
ax[2].set_title('core temp')
cb = fig.colorbar(hmp,ax=ax[2],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='°C',fontsize=fs)

hmp = ds_ww.ww_sa.mean('season').plot(x='lon',transform=crs,ax=ax[3],vmin=34.15,vmax=34.65,cmap='cmo.haline',add_colorbar=False)
ax[3].set_title('core asal')
cb = fig.colorbar(hmp,ax=ax[3],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='g/kg',fontsize=fs)

hmp = (ds_ww.ww_n2_int.mean('season')*1e3).plot(
            x='lon',transform=crs,ax=ax[4],vmin=0,vmax=1.5,cmap='cmo.tempo',add_colorbar=False)
ax[4].set_title('WW integrated n2')
cb = fig.colorbar(hmp,ax=ax[4],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label=r'$\times 10^{-3}s^{-1}$',fontsize=fs)

fig.text(-0.065,0.85,s=r'$\dot{var}$',fontsize=25,)

# define axis for 2nd row    
ax = ax1[5:]

ds_ww.ww_cp.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[0],vmin=-40,vmax=40,cmap='cmo.balance',add_colorbar=False)
ax[0].set_title('core depth')

hmp = ds_ww.thcc.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[1],vmin=-40,vmax=40,cmap='cmo.balance',add_colorbar=False)
ax[1].set_title('thickness')
cb = fig.colorbar(hmp,ax=ax[:2],shrink=0.7,aspect=asp*2,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.ww_ct.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[2],vmin=-0.5,vmax=0.5,cmap='cmo.curl',add_colorbar=False)
ax[2].set_title('core temp')
cb = fig.colorbar(hmp,ax=ax[2],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='°C',fontsize=fs)

hmp = ds_ww.ww_sa.mean('season').diff('lat').plot(x='lon',transform=crs,ax=ax[3],vmin=-0.3,vmax=0.3,cmap='cmo.delta',add_colorbar=False)
ax[3].set_title('core asal')
cb = fig.colorbar(hmp,ax=ax[3],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='g/kg',fontsize=fs)

hmp = (ds_ww.ww_n2_int.mean('season').diff('lat')*1e3).plot(
            x='lon',transform=crs,ax=ax[4],vmin=-0.3,vmax=0.3,cmap='cmo.diff',add_colorbar=False)
ax[4].set_title('WW mean n2')
cb = fig.colorbar(hmp,ax=ax[4],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label=r'$\times 10^{-3}s^{-1}$',fontsize=fs)

fig.text(-0.065,0.515,s=r'$\frac{d\,\dot{var}}{d\,lat}$',fontsize=32,)
    
# define ax for 3rd row

# add basic kwargs for circump plots
kwargs = dict(x='lon',transform=crs,vmin=0,add_colorbar=False,)

ax = ax1[-5:]

ds_ww.ww_cp.std('season').plot(**kwargs,ax=ax[0],vmax=40,cmap='cmo.amp',)

hmp = ds_ww.thcc.std('season').plot(**kwargs,ax=ax[1],vmax=40,cmap='cmo.amp',)
cb = fig.colorbar(hmp,ax=ax[:2],shrink=0.7,aspect=asp*2,location='bottom')
cb.set_label(label='dbar',fontsize=fs)

hmp = ds_ww.ww_ct.std('season').plot(**kwargs,ax=ax[2],vmax=0.5,cmap='Greens',)
cb = fig.colorbar(hmp,ax=ax[2],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='°C',fontsize=fs)

hmp = ds_ww.ww_sa.std('season').plot(**kwargs,ax=ax[3],vmax=0.2,cmap='BuGn',)
cb = fig.colorbar(hmp,ax=ax[3],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label='g/kg',fontsize=fs)

hmp = (ds_ww.ww_n2_int.std('season')*1e3).plot(**kwargs,ax=ax[4],vmax=1,cmap='Blues',)
cb = fig.colorbar(hmp,ax=ax[4],shrink=0.7,aspect=asp,location='bottom')
cb.set_label(label=r'$\times 10^{-3}s^{-1}$',fontsize=fs)

fig.text(-0.065,0.175,s=r'$\sigma(var)$',fontsize=25,)

fig.show()